In [22]:
import gzip
import numpy as np
from collections import defaultdict
from sklearn.metrics import mean_squared_error

In [23]:
def readGz(f):
    for l in gzip.open(f):
        yield eval(l)

In [28]:
from random import shuffle
data = []
for l in readGz("train.json.gz"):
    data.append(l)

print(data[0])
shuffle(data)
print(data[0])

#trainData, validData = np.array_split(data, 2)
trainData, validData = data, data[150000:]

{'rating': 5.0, 'businessID': 'B408037852', 'reviewText': "This is where i go to shop for gifts from my mom. She loves this stuff. Cna't get enough. I like that you can customize the items. Store is well alid out and shoppable.", 'userID': 'U093387342', 'reviewTime': 'Mar 24, 2013', 'categories': ["Women's Clothing Store", 'Fashion Accessories Store', 'Shoe Store'], 'reviewHash': 'R471510664', 'unixReviewTime': 1364143460}
{'rating': 3.0, 'businessID': 'B204140568', 'reviewText': "Eh, I understand that there aren't many locations in the city to have shows/conventions, but I've never had the best experiences here. Their food options are subpar and for the events I've gone to, I wouldn't be able to leave and come back. The space is decent though and it's accessible enough so ultimately the venue won't stop me from going to good events there.", 'userID': 'U491822196', 'reviewTime': 'Aug 11, 2013', 'categories': ['Convention Center', 'Auditorium'], 'reviewHash': 'R903330281', 'unixReviewTi

In [29]:
allRatings = []
userRatings = defaultdict(dict)
busRatings = defaultdict(dict)
#avgBusRatings = defaultdict(int)
globalAvg = 0.0
userIndex = defaultdict(int)
busIndex = defaultdict(int)

uCount = 0
bCount = 0

for d in trainData:
    user, bus, rating = d['userID'], d['businessID'], d['rating']
    allRatings.append(rating)
    globalAvg += rating
    userRatings[user][bus] = rating
    busRatings[bus][user] = rating
    if user not in userIndex:
        userIndex[user] = uCount
        uCount += 1
    if bus not in busIndex:
        busIndex[bus] = bCount
        bCount += 1

numUsers = len(userRatings) #storing the number of unique users in training set
numBus = len(busRatings) #storing the number of unique users in training set

print(len(userIndex))
print(numUsers)
print(len(busIndex))
print(numBus)

lenTData = len(allRatings)

globalAvg = globalAvg/lenTData

vallRatings = []
vuserRatings = defaultdict(dict)
vbusRatings = defaultdict(dict)

for d in validData:
    user, bus, rating = d['userID'], d['businessID'], d['rating']
    vallRatings.append(rating)
    vuserRatings[user][bus] = rating
    vbusRatings[bus][user] = rating
    
print(vallRatings[0])

18793
18793
21321
21321
3.0


In [30]:
alpha = 0;
betaU = defaultdict(int)
betaI = defaultdict(int)
lowDimSize = 10
gammaU = defaultdict(lambda: [1.0/lowDimSize for x in range(lowDimSize)])
gammaI = defaultdict(lambda: [1.0/lowDimSize for x in range(lowDimSize)])
lamda = 10

def resetVars(lSize, lamdanew):
    global alpha, betaU, betaI, gammaU, gammaI, lowDimSize, globalAvg, lamda
    lowDimSize = lSize
    alpha = 0.0
    lamda = lamdanew
    betaU = defaultdict(int)
    betaI = defaultdict(int)
    gammaU = defaultdict(lambda: np.array([1.0/lowDimSize for x in range(lowDimSize)]))
    gammaI = defaultdict(lambda: np.array([1.0/lowDimSize for x in range(lowDimSize)]))

In [31]:
predRatings = []
trainRatingTuples = [[d['userID'], d['businessID'], d['rating']] for d in trainData]
print(trainRatingTuples[0][0])

def predictValid():
    global predRatings
    global alpha, betaU, betaI, gammaU, gammaI, lowDimSize
    predRatings = []
    for d in validData:
        user, business = d['userID'], d['businessID']
        prediction = alpha + betaU[user] + betaI[business] + np.matmul(gammaU[user],np.transpose(gammaI[bus]))
        predRatings.append(prediction[0])
    #print(len(vallRatings))
    #print(len(predRatings))
    #print(predRatings[0])
    MSE_alpha = mean_squared_error(vallRatings, predRatings)
    return MSE_alpha

def trainLatentFactor(updateGammaU):
    global alpha, betaU, betaI, gammaU, gammaI, predRatings, lowDimSize, lamda
    
    #updating alpha
    
    newAlpha = 0.0
    for d in range(lenTData):
        user, business, rating = trainRatingTuples[d][0], trainRatingTuples[d][1], trainRatingTuples[d][2]
        gammaProd = np.matmul(gammaU[user], np.transpose(gammaI[business]))
        newAlpha += (userRatings[user][business] - betaU[user] - betaU[business] - gammaProd)
    newAlpha /= lenTData
    alpha = newAlpha
    
    #updating betaU
    for u in userRatings:
        tempBeta = 0.0
        busCount = 0
        for b in userRatings[u]:
            gammaProd = np.matmul(gammaU[u], np.transpose(gammaI[b]))
            tempBeta += (userRatings[u][b] - alpha - betaI[b] - gammaProd)
            busCount += 1
        tempBeta /= (lamda + busCount)
        betaU[u] = tempBeta
        
    #updating betaI
    for b in busRatings:
        tempBeta = 0.0
        uCount = 0
        for u in busRatings[b]:
            gammaProd = np.matmul(gammaU[u], np.transpose(gammaI[b]))
            tempBeta += (userRatings[u][b] - alpha - betaU[u] - gammaProd)
            uCount += 1
        tempBeta /= (lamda + uCount)
        betaI[b] = tempBeta
        
    if(updateGammaU):
        for u in userRatings:
            newGamma = np.zeros(shape=(1,lowDimSize))
            gammaProd = 0.0
            for b in userRatings[u]:
                temp = (userRatings[u][b] - alpha - betaU[u] - betaI[b])
                temp = temp*gammaI[b]
                newGamma = newGamma + temp
                gammaProd += np.matmul(gammaI[b], np.transpose(gammaI[b]))
            newGamma = newGamma/ (gammaProd + lamda)
            gammaU[u] = newGamma
    else:
        for b in busRatings:
            newGamma = np.zeros(shape=(1, lowDimSize))
            gammaProd = 0.0
            for u in busRatings[b]:
                newGamma = newGamma + (userRatings[u][b] - alpha - betaU[u] - betaI[b])*gammaU[u]
                gammaProd += np.matmul(gammaU[u], np.transpose(gammaU[u]))
            newGamma = newGamma / (gammaProd + lamda)
            gammaI[b] = newGamma
        
def train(i, k, lamdat):
    global lowDimSize, lamda
    lowDimSize = k
    lamda = lamdat
    for x in range(i):
        print("Iteration: "+str(x))
        if(x%2==0):
            trainLatentFactor(True)
        else:
            trainLatentFactor(False)
    print(lamdat)
    print(predictValid())

U491822196


In [14]:
dimSize = 2
lams = 6.0
for l in range(20):
    resetVars(dimSize, lams)
    print("Lamda value: "+str(lams))
    train(10, dimSize, lams)
    lams += 0.1
    print("Alpha value: " +str(alpha))

Lamda value: 6.0
6.0
0.597915970513
Alpha value: [[ 4.26334049]]
Lamda value: 6.1
6.1
0.597858713097
Alpha value: [[ 4.26311958]]
Lamda value: 6.199999999999999
6.199999999999999
0.597812136062
Alpha value: [[ 4.2628999]]
Lamda value: 6.299999999999999
6.299999999999999
0.597775603151
Alpha value: [[ 4.26268129]]
Lamda value: 6.399999999999999
6.399999999999999
0.597748554625
Alpha value: [[ 4.26246365]]
Lamda value: 6.499999999999998
6.499999999999998
0.597730497446
Alpha value: [[ 4.26224686]]
Lamda value: 6.599999999999998
6.599999999999998
0.597720995991
Alpha value: [[ 4.26203085]]
Lamda value: 6.6999999999999975
6.6999999999999975
0.597719663417
Alpha value: [[ 4.26181556]]
Lamda value: 6.799999999999997
6.799999999999997
0.597726153734
Alpha value: [[ 4.26160094]]
Lamda value: 6.899999999999997
6.899999999999997
0.597740154709
Alpha value: [[ 4.26138697]]
Lamda value: 6.9999999999999964
6.9999999999999964
0.597761381652
Alpha value: [[ 4.2611736]]
Lamda value: 7.099999999999996


KeyboardInterrupt: 

In [36]:
#resetVars(2, 4.5)
#train(60, 2, 4.5)

4.5
0.609137951299


In [41]:
resetVars(1, 4.5)
train(100, 1, 4.5)

4.5
0.454873279147


In [32]:
resetVars(1, 6.7)
train(100, 1, 6.7)

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteration: 10
Iteration: 11
Iteration: 12
Iteration: 13
Iteration: 14
Iteration: 15
Iteration: 16
Iteration: 17
Iteration: 18
Iteration: 19
Iteration: 20
Iteration: 21
Iteration: 22
Iteration: 23
Iteration: 24
Iteration: 25
Iteration: 26
Iteration: 27
Iteration: 28
Iteration: 29
Iteration: 30
Iteration: 31
Iteration: 32
Iteration: 33
Iteration: 34
Iteration: 35
Iteration: 36
Iteration: 37
Iteration: 38
Iteration: 39
Iteration: 40
Iteration: 41
Iteration: 42
Iteration: 43
Iteration: 44
Iteration: 45
Iteration: 46
Iteration: 47
Iteration: 48
Iteration: 49
Iteration: 50
Iteration: 51
Iteration: 52
Iteration: 53
Iteration: 54
Iteration: 55
Iteration: 56
Iteration: 57
Iteration: 58
Iteration: 59
Iteration: 60
Iteration: 61
Iteration: 62
Iteration: 63
Iteration: 64
Iteration: 65
Iteration: 66
Iteration: 67
Iteration: 68
Iteration: 69
Iteration: 70
Iteration: 71
It

In [20]:
def ratingPredFile(inputFile, outputFile):
    global alpha, betaU, betaI, gammaU, gammaI, lowDimSize
    predictions = open(outputFile, 'w')
    for l in open(inputFile):
        if l.startswith("userID"):
            predictions.write(l)
            continue
        user,business = l.strip().split('-')
        prediction = 0.0
        prediction = prediction + alpha
        if user in betaU and business in betaI:
            prediction += (betaU[user] + betaI[business] + np.matmul(gammaU[user], np.transpose(gammaI[business])))
            #print(prediction)
        elif user in betaU:
            prediction += betaU[user]
        elif business in betaI:
            prediction += betaI[business]
        
        predictions.write(user + '-' + business + "," + str(prediction[0][0])+"\n")

ratingPredFile("pairs_Rating.txt","predictions_Rating.txt")
print("done")

done


In [ ]:
predictValid()

In [26]:
np.matmul(gammaU['U455762636'],np.transpose(gammaI['B408037852']))[0][0] + betaI['B408037852']

array([[ 0.05303588]])